# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'llama3.2'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [13]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [9]:
def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape

cnn = Website("https://cnn.com")
cnn.links

['https://edition.cnn.com',
 'https://edition.cnn.com/us',
 'https://edition.cnn.com/world',
 'https://edition.cnn.com/politics',
 'https://edition.cnn.com/business',
 'https://edition.cnn.com/health',
 'https://edition.cnn.com/entertainment',
 'https://edition.cnn.com/style',
 'https://edition.cnn.com/travel',
 'https://edition.cnn.com/sports',
 'https://edition.cnn.com/science',
 'https://edition.cnn.com/climate',
 'https://edition.cnn.com/weather',
 'https://edition.cnn.com/world/europe/ukraine',
 'https://edition.cnn.com/world/middleeast/israel',
 'https://edition.cnn.com/games',
 'https://edition.cnn.com/us',
 'https://edition.cnn.com/world',
 'https://edition.cnn.com/politics',
 'https://edition.cnn.com/business',
 'https://edition.cnn.com/health',
 'https://edition.cnn.com/entertainment',
 'https://edition.cnn.com/style',
 'https://edition.cnn.com/travel',
 'https://edition.cnn.com/sports',
 'https://edition.cnn.com/science',
 'https://edition.cnn.com/climate',
 'https://edition

In [14]:
get_links("https://cnn.com")

{'description': '',
 'logo': {'url': 'https://media1.pstatic.net/smartphone/2018/11/e7b0da2a79ca5c15e45c4c02b2f83b7b.png?image=FLC&height=50&size=102x102',
  'width': 102,
  'height': 102},
 'version': {},
 'url': ''}

In [15]:
get_links("https://anthropic.com")

{'links': [{'type': 'About page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'Careers/Jobs page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'Solutions/Coding Solutions page',
   'url': 'https://www.anthropic.com/solutions/coding'},
  {'type': 'Customer Support page',
   'url': 'https://www.anthropic.com/solutions/customer-support'},
  {'type': 'Enterprise page', 'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'News/Claude News page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'News Core Views on AI Safety page',
   'url': 'https://www.anthropic.com/news/core-views-on-ai-safety'},
  {'type': 'News Constitutional AI Harmlessness from AI Feedback page',
   'url': 'https://www.anthropic.com/news/constitutional-ai-harmlessness-from-ai-feedback'},
  {'type': 'Pricing page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'Resources API page', 'url': 'https://docs.

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [24]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        try:
            r = Website(link["url"]).get_contents()
            result += f"\n\n{link['type']}\n"
            result += r
        except:
            pass
    return result

In [25]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com'}, {'type': 'About page', 'url': 'https://docs.anthropic.com/|'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'projects/person', 'url': 'https://claude.ai/'}, {'type': 'solutions', 'url': 'https://support.anthropic.com/solutions/coding'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
API
Solutions
Research
Commitments
Learn
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Claude 3.7 Sonnet and Claude Code
Introducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model

In [34]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [28]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [29]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'About page', 'url': 'https://www.anthropic.com'}, {'type': 'Company page', 'url': 'https://www.anthropic.com/'}, {'type': 'Careers/Jobs page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'News section', 'url': 'https://www.anthropic.com/news'}, {'type': 'Claude AI homepage', 'url': 'https://claude.ai/'}, {'type': 'Research section', 'url': 'https://www.anthropic.com/research'}, {'type': "Research paper on Claude's latest sonnet", 'url': 'https://www.anthropic.com/news/claude-3-7-sonnet'}, {'type': 'Visible Extended Thinking research paper', 'url': 'https://www.anthropic.com news/visible-extended-thinking'}, {'type': 'Enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'Learning platform', 'url': 'https://learn.anthropic.com/'}, {'type': 'Trust section', 'url': 'https://trust.anthropic.com'}, {'type': 'Supported countries page', 'url': 'https://www.anthropic.com/supported-countries'}, {'type': 'X.com link for Anthropic A

"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nClaude 3.7 Sonnet and Claude Code\nIntroducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.\nRead the announcement\nModel details\nClaude 3.7 Sonnet\nResearch insights\nClaude's extended thinking\nOur work\nProduct\nClaude for Enterpris

In [30]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'About page', 'url': 'https://www.anthropic.com'}, {'type': 'Company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'Careers/Jobs page', 'url': 'https://www.anthropic.com/careers'}]}


# Brochure: Anthropic

## About Us

Anthropic is an AI safety and research company dedicated to building reliable, interpretable, and steerable AI systems that people can rely on. Our mission is to harness the potential of AI while ensuring its safe deployment.

### Our Purpose

We believe that AI will have a profound impact on the world and are committed to researching and developing systems that can reap the benefits while minimizing the risks. We aim to build AI systems that are not only intelligent but also transparent, accountable, and beneficial.

## Company Values

* **Safety as a Science**: We treat AI safety as a systematic science, applying research insights to our products and feeding those results back into our research.
* **Reliability and Interpretability**: We prioritize building reliable, interpretable, and steerable AI systems that can be trusted by users.
* **Collaboration**: We collaborate with academia, industry partners, and researchers to share knowledge and drive progress in AI safety.

## Our Technology

Our flagship product, Claude, is a cutting-edge AI model designed for reliability, interpretability, and steerability. With the launch of Claude 3.7 Sonnet, our most intelligent AI model yet, we are pushing the boundaries of what is possible with hybrid reasoning models.

### Key Features:

* **Hybrid Reasoning Models**: Our unique approach combines strengths from both symbolic and connectionist AI to deliver unparalleled performance.
* **Advanced Safety Techniques**: We apply a variety of safety techniques, including feedback mechanisms, to ensure our systems align with human values.

## Customers and Partners

Anthropic collaborates with leading organizations across industries to bring reliable, beneficial AI solutions to market. Our customers value our commitment to safety, transparency, and interpretability.

### Industries:

* **Financial Services**: We help banks and financial institutions build AI-powered applications that ensure compliance with regulations.
* **Healthcare**: Our partnerships enable healthcare providers to develop patient-centric AI systems that advance medical research and treatment.

## Careers

Join us in shaping the future of AI. Anthropic offers opportunities for talented individuals from diverse backgrounds to contribute to our cutting-edge projects.

### Open Roles:

* **Research Scientists**
* **Engineers**
* **Data Analysts**
* **Software Developers**

## News and Updates

Stay up-to-date with the latest breakthroughs and achievements from Anthropic by following our news blog and social media channels.

### Blog:
[Link to company blog]

### Twitter:
[Link to company Twitter handle]

### LinkedIn:
[Link to company LinkedIn profile]

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [32]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'About page', 'url': 'https://claude.ai/'}, {'type': 'Company page', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Careers/Jobs page', 'url': 'https://status.anthropic.com/'}, {'type': 'Products/Accomplishments (Claude AI Sonnet)', 'url': 'https://www.anthropic.com/research#entry:8@1:url'}, {'type': 'Research/Economic-Index', 'url': 'https://docs.anthropic.com/'}, {'type': 'Solutions/Agents', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'Solutions/Coding', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'Solutions/Customer-Support', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'Learning Resource (Anthropic University)', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Blog News page', 'url': 'https://www.anthropic.com/news'}, {'type': 'Blog Customers page', 'url': 'https://support.anthropic.com/customers'}, {'type': 'About Trust page', 'url': 'https://trust.anthropic.com'}, {'type'

**Anthropic Brochure**
======================

**About Us**
------------

Anthropic is an AI safety and research company based in San Francisco, dedicated to building frontier AI models that put safety at the forefront. Our interdisciplinary team of experts has experience across machine learning, physics, policy, and product development.

**Our Mission**
----------------

To develop reliable and beneficial AI systems that prioritize safety, security, and transparency. We strive to be a leader in the AI research community, pushing the boundaries of what is possible while ensuring our work minimizes risks.

**Our Products**
----------------

* **Claude**: Our flagship model, which combines advanced reasoning, vision analysis, and code generation capabilities. Claude is designed for developers, educators, and users seeking to harness the power of AI.
* **Claude 3.7 Sonnet**: The latest advancement in our Claude model family, offering unparalleled performance, speed, and safety.

**Our Commitments**
------------------

* **Transparency**: We prioritize open communication about our research, methods, and results.
* **Security and Compliance**: Our products and services are designed to meet the highest security and compliance standards.
* **Responsible Scaling Policy**: We strive to ensure that our growth does not compromise safety or the environment.

**Join Our Mission**
-------------------

Explore available positions on our [careers page](https://anthropic.com/careers). Collaborate with us and contribute to shaping a future of AI that prioritizes humanity and responsibility.

**Stay Informed**
----------------

Read our latest research papers, publications, and announcements at our [research page](https://anthropic.com/research).

Contact Us
------------

Learn more about Claude or our mission by visiting our website at [www.anthropice.com](http://www.anthropice.com) or reaching out to us directly.

**Anthropic - Building a Safer Tomorrow**
-----------------------------------------------

In [35]:
# Try changing the system prompt to the humorous version when you make the Brochure for Anthropic:

stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'About page', 'url': 'https://www.anthropic.com/company'}, {'type': 'Claude AI homepage', 'url': 'https://claude.ai/'}, {'type': 'Company page', 'url': 'https://www.anthropic.com/'}, {'type': 'Careers/Jobs page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Transparency page', 'url': 'https://www.anthropic.com transparency'}, {'type': 'Responsible scaling policy', 'url': 'https://www.anthropic.com/responsible-scaling-policy'}, {'type': 'Solutions/Agents page', 'url': 'https://anthropic.solutions agents/agents/'}, {'type': 'Economic Index page', 'url': 'https://anthropic.solutions/economic-index='}, {'type': 'Education/E learning page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'API Documentation', 'url': 'https://docs.anthropic.com/'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


**Anthropic: Building Reliable AIs for a Better World**

[Cover Image: A stylized illustration of a compass with a glowing needle, symbolizing guidance and direction]

Welcome to Anthropic, an AI safety and research company dedicated to building systems that people can rely on. We're driven by a shared purpose: to make decisions that maximize positive outcomes for humanity in the long run.

**Our People**
We're a team of 250+ talented individuals from diverse backgrounds, united by our passion for developing reliable and trustworthy AI systems. Our team includes researchers, engineers, policy experts, business leaders, and operational leaders who bring a wealth of experience to our work.

[Image: A group photo of Anthropic team members from various disciplines]

**Our Values**

At Anthropic, we value:

*   **Acting for the global good**: We strive to make decisions that maximize positive outcomes for humanity in the long run. From AI system development to research and policy-making, our aim is to create technology that benefits society as a whole.
*   *Shade and light*: We recognize both the potential risks and rewards of AI. By exploring both sides, we can better understand the opportunities and challenges ahead while promoting safe and responsible innovation.
*   **Caring for our users**: As a company, we define 'users' broadly – encompassing not only customers but also policymakers, Ants (AI systems), and anyone impacted by our technology. We prioritize generosity and kindness in all interactions, striving to cultivate empathy and understanding.

**What We Do**

We're an interdisciplinary team working at the intersection of AI research, policy, and operations. Our work involves:

*   Frontier AI research across various modalities
*   Exploring novel safety research areas, such as interpretability and RL from human feedback
*   Developing practical tools like Claude that benefit businesses, nonprofits, and civil society groups worldwide

**Impact**

At Anthropic, we aim to make a positive impact on the world. By:

*   Conducting regular research and sharing findings with policymakers, academia, and industry stakeholders
*   Collaborating with civil society, government, academia, nonprofits, and industry partners to promote safety across the AI landscape
*   Contributing to shaping the development of AI regulations, standards, and best practices

Join us on this journey. Together, let's create AI systems that people can rely on – making a positive impact for generations to come.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>